In [1]:
MARKET = 'SNP'

In [2]:
import sys
import pathlib
import numpy as np
import pandas as pd
import yaml
import asyncio

from ib_insync import IB, util, Option, MarketOrder, Contract
from typing import Callable, Coroutine, Union

In [3]:
# Specific to Jupyter. Will be ignored in IDE / command-lines
import IPython as ipy
if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    import nest_asyncio
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None
    
    THIS_FOLDER = '' # Dummy for jupyter notebook's current folder

In [4]:
# Get capability to import programs from `asyncib` folder
cwd = pathlib.Path.cwd() # working directory from where python was initiated
DATAPATH = cwd.joinpath('data', MARKET.lower()) # path to store data files
LOGFILE = DATAPATH.joinpath('temp.log') # path to store log files

IBPATH = cwd.parent.parent.joinpath('asyncib') # where ib programs are stored

# append IBPATH to import programs.
if str(IBPATH) not in sys.path:  # Convert it to string!
    sys.path.append(str(IBPATH))
    
IBDATAPATH = IBPATH.joinpath('data', MARKET.lower())

In [5]:
# Get the host, port, cid
from engine import Vars

ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID

In [6]:
# Get the pickle files
from os import listdir
fs = listdir(DATAPATH)

files = [f for f in fs if f[-4:] == '.pkl']
for f in files:
    exec(f"{f.split('.')[0]} = pd.read_pickle(DATAPATH.joinpath(f))")
files

['df_chains.pkl',
 'df_ohlcs.pkl',
 'df_opts.pkl',
 'df_opt_margins.pkl',
 'df_opt_prices.pkl',
 'df_symlots.pkl',
 'df_unds.pkl',
 'df_und_margins.pkl',
 'df_und_prices.pkl',
 'qopts.pkl']

# Refactoring dfrq (copied from)

In [7]:
from support import Timer, Vars
from engine import qualify, executeAsync, margin, save_df
from dfrq import quick_pf, get_pnl

In [11]:
%%time
ibp = Vars(MARKET.upper())  # IB Parameters from var.yml

HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID

LOGPATH = pathlib.Path.cwd().joinpath(THIS_FOLDER, "data", "log")
DATAPATH = pathlib.Path.cwd().joinpath(THIS_FOLDER, "data", MARKET.lower())

# * SETUP LOGS AND CLEAR THEM
LOGFILE = LOGPATH.joinpath(MARKET.lower() + "_dfrq.log")
util.logToFile(path=LOGFILE, level=30)
with open(LOGFILE, "w"):
    pass

# ... start the time
dfrq_time = Timer('dfrqs')
dfrq_time.start()

# * GET PNL AND PORTFOLIO

# ... portfolio
with IB().connect(ibp.HOST, ibp.PORT, ibp.CID) as ib:
    df_pf = quick_pf(ib)
    ib.disconnect()
    IB().waitOnUpdate(timeout=ibp.FIRST_XN_TIMEOUT)

# ... pnl (after a break to avoid first-connection timeout issue)
with IB().connect(ibp.HOST, ibp.PORT, ibp.CID) as ib:
    pnl_dict = ib.run(get_pnl(ib, MARKET))

# * GET MARGINS CONSUMED BY PORTFOLIO
pf_raw_cts = [Contract(conId=c) for c in df_pf.conId]

# .qualify portfolio contracts
with IB().connect(HOST, PORT, CID) as ib:
    pf_cts = ib.run(qualify(ib, pf_raw_cts))

# .get orders and make cos
df_pf = df_pf.assign(contract = pf_cts)
df_pf = df_pf.assign(order=[MarketOrder('SELL', abs(p))
                                    if p > 0 else MarketOrder('BUY', abs(p))
                                    for p in df_pf.position])

cos = [(c, o) for c, o in zip(df_pf.contract, df_pf.order)]

# .get margins
with IB().connect(HOST, PORT, CID) as ib:
    df_pfm = ib.run(executeAsync(ib=ib, algo=margin, cts=cos,
                                             CONCURRENT=200, TIMEOUT=5, 
                                             post_process=save_df))

df_pf = df_pf.set_index('conId').\
            join(df_pfm[['conId', 'margin', 'comm']].\
                 set_index('conId')).reset_index().\
                    drop('order', 1)



# .get gross position (long/short commitment)
df_pf = df_pf.assign(grosspos=np.where(df_pf.secType == 'OPT', df_pf.strike * df_pf.position * df_pf.lot, 
                                       df_pf.mktPrice * df_pf.position * df_pf.lot))

# .get rom (return-on-margin) and rog (return-on-grosspos)
df_pf = df_pf.assign(rog=df_pf.unPnL/(df_pf.grosspos).apply(abs), 
             rom=df_pf.unPnL/(df_pf.margin).apply(abs))


Done margin for ['SLB1113C16.5..', 'SLB0115P17.5..'] 97 out of 97. Pending []
Wall time: 5.03 s


## Let us analyze the returns

In [45]:
# .putting it all together
sym_margins = df_pf.groupby('symbol').margin.apply(sum)
sym_grosspos = df_pf.groupby('symbol').grosspos.apply(sum)
sym_unPnL = df_pf.groupby('symbol').unPnL.apply(sum)

df_returns = pd.concat([sym_margins, sym_grosspos, sym_unPnL], axis=1).sort_values('unPnL', ascending=True)
df_returns = df_returns.assign(rog = df_returns.unPnL/(df_returns.grosspos.apply(abs)),
                  rom = df_returns.unPnL/(df_returns.margin.apply(abs)))

In [56]:
df_pf[df_pf.symbol == 'GM']

,conId,secType,symbol,expiry,strike,right,position,lot,mktPrice,mktVal,avgCost,unPnL,rePnL,contract,margin,comm,grosspos,rog,rom
48,333597631,OPT,GM,20210115,30.0,P,1.0,100,0.248937,24.89,630.7938,-605.9,0.0,"Contract(secType='OPT', conId=333597631, symbo...",-24.89,1.58635,3000.0,-0.201967,-24.34311


In [55]:
df_returns.sort_values('rom', ascending=True)

,margin,grosspos,unPnL,rog,rom
symbol,,,,,
GM,-24.89,3000.000000,-605.90,-0.201967,-24.343110
XME,-75.89,8400.000000,-1026.22,-0.122169,-13.522467
MDT,-201.99,28500.000000,-2684.14,-0.094180,-13.288480
MCD,-201.46,36000.000000,-2589.43,-0.071929,-12.853321
CVX,587.44,21255.999760,-7108.11,-0.334405,-12.100146
DOW,-47.88,4000.000000,-480.91,-0.120228,-10.044069
COF,-164.55,7250.000000,-1431.25,-0.197414,-8.697964
CAT,-348.63,36000.000000,-2363.05,-0.065640,-6.778103
COP,-391.21,26415.000925,-2480.77,-0.093915,-6.341275
